# UCI ML Repository: Adult Income Data Set
---
### Case Optimum - Paulo Henrique Spada de Moura (Julho/2020)
### Aplicação de modelo de Regressão Logística para predição de renda anual

* Importação das bibliotecas básicas (pandas, numpy, etc). Os pacotes do *sklearn* para aplicação do modelo de Regressão Logística serão importados posteriormente:

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

* Importação do data set através do repositório da UCI e inserção dos nomes das colunas para identificação dos atributos

In [ ]:
colunas = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship',
                'race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
file = ('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')
adult = pd.read_csv(file,names=colunas)

In [ ]:
adult.head()

In [ ]:
adult.info()

A função abaixo foi criada para apresentar os valores únicos por atributo em nosso data set. Com ela, podemos checar se existem valores "estranhos" à coluna e avaliarmos como substituí-los, pois a 'não existência' de valores faltantes em uma base com tantos registros deve despertar alguma desconfiança:

In [ ]:
def check_unicos():
        for col in colunas:
            print(adult[col].unique())
check_unicos()

Com a aplicação da função acima, observamos que os valores categóricos do data set apresentam espaçamento à esquerda, no início. Para retirarmos esses espaços, aplicamos:

In [ ]:
#Os valores categóricos do dataset estão com um espaçamento à esquerda. Retirando esses espaços:

for col in ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','income']:
    adult[col] = adult[col].str.lstrip()

In [ ]:
check_unicos()

Após a correção dos espaçamentos, podemos identificar também a terminologia "?" para valores não existentes (por esse motivo não era indicados valores nulos na prévia análise do data set). Nesse passo, serão efetivamente procador por valores nulos (*NaN*):

In [ ]:
adult[adult == '?'] = np.nan

Conferindo novamente a existência de valores nulos:

In [ ]:
adult.isnull().sum()

Podemos observar que pouco menos de 5% dos registros apresentam valores únicos. Por representar uma parcela muito pequena do data set, as linhas que apresentam esses valores serão **removidas por completo**:

In [ ]:
adult = adult.dropna()
adult.info()

## Aplicação da Regressão Logística

No contexto da utilização do algoritmo, trataremos de uma base com parâmetros um pouco mais simples, na qual a aplicação da Regressão Logística, por mais que os dados não sejam exatamente separáveis de maneira linear, será suficiente para realizar as devidas predições e, a posteriori, até mesmo indicar se a aplicação de um modelo mais complexo (por exemplo, Random Forest) retornará uma acuracidade maior ou não. Em suma, será aplicada a Regressão Logística para âmbito de estudo e diversificação dos modelos.

* Definição dos dataframes com as **variáveis independentes** e a **variável target (*'income'*)** para construção do modelo:

In [ ]:
X = adult.drop(['income'],axis=1)
y = adult['income']

In [ ]:
X.head()

In [ ]:
y.head()

* Separação das bases de treino e teste, seguindo a recomendação da documentação do data set, que sugere o *split* **70% treino** e **30% teste**:

In [ ]:
# Importação das bibliotecas sklearn para aplicação do modelo

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

* Correção do *encoding* para as variáveis categóricas:

In [ ]:
from sklearn import preprocessing

categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
for cat in categoricas:
    le = preprocessing.LabelEncoder()
    X_train[cat] = le.fit_transform(X_train[cat])
    X_test[cat] = le.transform(X_test[cat])

* **Dimensionamento das variáveis independentes** (*Feature Scaling* - normalização dos intervalos das variáveis independentes):

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

In [ ]:
X_train.head()

- Passo 1) Utilização do modelo de regressão logística com **TODAS** as variáveis:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Acuracidade da regressão logística (todas as variáveis): {} ' .format(accuracy_score(y_test, y_pred)))

- Passo 2) Aplicação do algoritmo de **análise de componentes principais (PCA)** para utilização das variáveis mais relevantes. Utilizando a função "*explained_variance_ratio_*", é indicada a **proporção da variância** para a inserção de cada componente principal:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_train = pca.fit_transform(X_train)
pca.explained_variance_ratio_

Na *array* resultante do passo acima,observados a **parcela de variância** pela qual cada variável é responsável no modelo. A seguir, faremos os testes **reduzindo a dimensionalidade**, de acordo com as variâncias de menor significância para avaliação da **acuracidade** do modelo de regressão logística:

- Passo 2.1) Eliminação da **última variável** ("*native-country*" - 2,75% de variância): 

In [ ]:
X = adult.drop(['income','native-country'], axis=1)
y = adult['income']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
for cat in categoricas:
        le = preprocessing.LabelEncoder()
        X_train[cat] = le.fit_transform(X_train[cat])
        X_test[cat] = le.transform(X_test[cat])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Acuracidade da regressão logística (13 variáveis): {} '. format(accuracy_score(y_test, y_pred)))

- Passo 2.2) Eliminação das **duas últimas variáveis** ("*native-country*" e "*hours-per-week*" - ~7% de variância):

In [ ]:
X = adult.drop(['income','native-country', 'hours-per-week'], axis=1)
y = adult['income']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
for cat in categoricas:
        le = preprocessing.LabelEncoder()
        X_train[cat] = le.fit_transform(X_train[cat])
        X_test[cat] = le.transform(X_test[cat])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)


print('Acuracidade da regressão logística (12 variáveis): {} '. format(accuracy_score(y_test, y_pred)))

- Passo 2.3) Eliminação das **três últimas variáveis** ("*native-country*", "*hours-per-week*" e "*capital-loss*" - ~12% de variância): 

In [ ]:
X = adult.drop(['income','native-country', 'hours-per-week', 'capital-loss'], axis=1)
y = adult['income']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
for cat in categoricas:
        le = preprocessing.LabelEncoder()
        X_train[cat] = le.fit_transform(X_train[cat])
        X_test[cat] = le.transform(X_test[cat])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)


print('Acuracidade da regressão logística (11 variáveis): {} '. format(accuracy_score(y_test, y_pred)))

Considerando o problema em questão, por contarmos com uma grande quantidade de variáveis independentes (grande número de dimensões para o modelo), escolheremos uma quantidade de dimensões que possam **explicar de maneira significativa uma grande parcela da variância (pelo menos 90%)**:

In [ ]:
X = adult.drop(['income'], axis=1)
y = adult['income']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)


categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
for cat in categoricas:
        le = preprocessing.LabelEncoder()
        X_train[cat] = le.fit_transform(X_train[cat])
        X_test[cat] = le.transform(X_test[cat])


X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)


pca= PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
dim = np.argmax(cumsum >= 0.90) + 1
print('O número de dimensões necessárias para preservar 90% de variância é',dim)

Dessa forma, analisando a acuracidade dos modelos de regressão logística, usaremos as **12 dimensões mais relevantes**, retornando uma acuracidade de, aproximadamente, **81,33%**.

A determinação das dimensões com PCA também pode ser auxiliada com recursos gráficos, assim como demonstrado abaixo. Aqui, na curva cumulativa da variância de acordo com a quantidade de dimensões, identificaremos o ponto aproximado de "cotovelo":

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,14,1)
plt.xlabel('Quantidade de dimensões')
plt.ylabel('Variância acumulada')
plt.show()

Pelo gráfico, podemos observar que a quantidade de dimensões para representação de **90% de variância** ocorre entre **11~12 dimensões**.